# Flatten tiles

# table of content
1) [Splitting](#splitting)
2) [Flattening of stripes](#flattening-of-stripes)
3) [Manual cleaning](#manual-cleaning)
4) [Post-process](#post-process)

### Dependencies and general utils

In [4]:
import os
import numpy as np
import laspy
import pdal
import json
import scipy
import copy
from tqdm import tqdm
from time import time
import itertools

### Inputs

In [34]:
src_original = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\quadric_interp\Barmasse_2025_AllScans_Raw_Sub2cm.laz"
tile_size = 500
overlap = 50
grid_size = 5
stripe_width = 10
method='quadric'
do_save_floor=True
do_keep_existing_flatten = False

### Utils

In [35]:
# utils

def tilling_pdal(src_input, src_target, tile_size, overlap=0, verbose=True):
    """
    Tile the input LiDAR file into square tiles using PDAL and store them in the destination folder.

    Args:
        - verbose (bool): Whether to print verbose status updates.

    Returns:
        - None: Splits the input file into tiles and saves them in the destination folder.
    """

    print(f"Start tilling (with overlap = {overlap}m)...")
    os.makedirs(src_target, exist_ok=True)
    
    # compute the estimate number of tiles
    if verbose:
        print("Computing the bounds...")
    original_file = laspy.read(src_input)
    x_min = original_file.x.min()
    x_max = original_file.x.max()
    y_min = original_file.y.min()
    y_max = original_file.y.max() 
    if verbose:
        print('Done!')

    output_pattern = os.path.join(
        src_target, 
        os.path.basename(src_input).split('.')[0] + "_tile_#.laz",
        )

    x_steps = int((x_max - x_min) / tile_size) + 1
    y_steps = int((y_max - y_min) / tile_size) + 1
    combinations = list(itertools.product(range(x_steps), range(y_steps)))
    list_bounds = []
    for (i,j) in combinations:
        x0 = x_min + i * tile_size - overlap
        x1 = x_min + (i + 1) * tile_size + overlap
        y0 = y_min + j * tile_size - overlap
        y1 = y_min + (j + 1) * tile_size + overlap

        bounds = f"([{x0},{x1}],[{y0},{y1}])"
        list_bounds.append(bounds)

    pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": src_input,
                "extra_dims": "id_point=uint32"
            },
            {
                "type": "filters.crop", 
                "bounds": list_bounds
            },
            {
                "type": "writers.las", 
                "filename": output_pattern, 
                'extra_dims': "id_point=uint32"
            }
        ]
    }
    
    if verbose:
        print("Creation of the tiles (might take a few minutes)")
    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    pipeline.execute()
    if verbose:
        print("Process done")


def tilling(src_input, src_target, tile_size, overlap=0, verbose=True):
    """
    Crops a LAS/LAZ file into tiles using laspy directly (preserves id_point and coordinates).
    """
    os.makedirs(src_target, exist_ok=True)

    las = laspy.read(src_input)
    x_min, x_max = las.x.min(), las.x.max()
    y_min, y_max = las.y.min(), las.y.max()

    x_steps = int((x_max - x_min) / tile_size) + 1
    y_steps = int((y_max - y_min) / tile_size) + 1

    combinations = list(itertools.product(range(x_steps), range(y_steps)))
    
    print("Creation of tiles:")
    for _, (ix, iy) in tqdm(enumerate(combinations), total=len(combinations)):
    # for ix in range(x_steps):
    #     for iy in range(y_steps):
        x0 = x_min + ix * tile_size - overlap
        x1 = x_min + (ix + 1) * tile_size + overlap
        y0 = y_min + iy * tile_size - overlap
        y1 = y_min + (iy + 1) * tile_size + overlap

        mask = (
            (las.x >= x0) & (las.x <= x1) &
            (las.y >= y0) & (las.y <= y1)
        )
        if not np.any(mask):
            continue

        # ✅ Create new file with proper header scale/offset
        header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
        header.offsets = las.header.offsets
        header.scales = las.header.scales

        # ✅ Copy CRS if any
        if hasattr(las.header, "epsg") and las.header.epsg is not None:
            header.epsg = las.header.epsg

        tile = laspy.LasData(header)
        tile.points = las.points[mask]

        tile_filename = os.path.join(
            src_target,
            f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
        )
        tile.write(tile_filename)


def stripes_file(src_input_file, src_output, dims, do_keep_existing=False, verbose=True):
    """
    Crops a LAS/LAZ file into tiles using laspy directly (preserves id_point and coordinates).
    """
    [tile_size_x, tile_size_y] = dims
    las = laspy.read(src_input_file)
    os.makedirs(src_output, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(las.x), np.max(las.x), np.min(las.y), np.max(las.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)

    combinations = list(itertools.product(x_edges, y_edges))
    # for (x0, y0) in combinations:
    num_skipped = 0
    for id_stripe, (x0, y0) in tqdm(enumerate(combinations), total=len(combinations)):
        x1 = x0 + tile_size_x
        y1 = y0 + tile_size_y

        mask = (
            (las.x >= x0) & (las.x <= x1) &
            (las.y >= y0) & (las.y <= y1)
        )
        if not np.any(mask):
            num_skipped += 1
            continue

        # ✅ Create new file with proper header scale/offset
        header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
        header.offsets = las.header.offsets
        header.scales = las.header.scales

        # ✅ Copy CRS if any
        if hasattr(las.header, "epsg") and las.header.epsg is not None:
            header.epsg = las.header.epsg

        tile = laspy.LasData(header)
        tile.points = las.points[mask]

        tile_filename = os.path.join(
            src_output,
            f"{os.path.splitext(os.path.basename(src_input_file))[0]}_stripe_{id_stripe - num_skipped}.laz"
        )
        tile.write(tile_filename)







    # os.makedirs(src_target, exist_ok=True)

    # las = laspy.read(src_input)
    # x_min, x_max = las.x.min(), las.x.max()
    # y_min, y_max = las.y.min(), las.y.max()

    # x_steps = int((x_max - x_min) / tile_size) + 1
    # y_steps = int((y_max - y_min) / tile_size) + 1

    # combination = list(itertools.product(range(x_steps), range(y_steps)))
    
    # print("Creation of tiles:")
    # for _, (ix, iy) in tqdm(enumerate(combination)):
    # # for ix in range(x_steps):
    # #     for iy in range(y_steps):
    #     x0 = x_min + ix * tile_size - overlap
    #     x1 = x_min + (ix + 1) * tile_size + overlap
    #     y0 = y_min + iy * tile_size - overlap
    #     y1 = y_min + (iy + 1) * tile_size + overlap

    #     mask = (
    #         (las.x >= x0) & (las.x <= x1) &
    #         (las.y >= y0) & (las.y <= y1)
    #     )
    #     if not np.any(mask):
    #         continue

    #     # ✅ Create new file with proper header scale/offset
    #     header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
    #     header.offsets = las.header.offsets
    #     header.scales = las.header.scales

    #     # ✅ Copy CRS if any
    #     if hasattr(las.header, "epsg") and las.header.epsg is not None:
    #         header.epsg = las.header.epsg

    #     tile = laspy.LasData(header)
    #     tile.points = las.points[mask]

    #     tile_filename = os.path.join(
    #         src_target,
    #         f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
    #     )
    #     tile.write(tile_filename)


def stripes_file_pdal(src_input_file, src_output, dims, do_keep_existing=False, verbose=True):
    [tile_size_x, tile_size_y] = dims
    laz = laspy.read(src_input_file)
    os.makedirs(src_output, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)

    if verbose:
        print("Computing the bounds...")

    list_bounds = []
    combinations = list(itertools.product(x_edges, y_edges))
    for (x0, y0) in combinations:
        # for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
        x1 = x0 + tile_size_x
        y1 = y0 + tile_size_y
        bounds = f"([{x0},{x1}],[{y0},{y1}])"
        
        list_bounds.append(bounds)

    output_pattern = os.path.join(
            src_output, 
            os.path.basename(src_input_file).split('.')[0] + "_stripe_#.laz",
            )
    
    pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": src_input_file,
                "spatialreference": "EPSG:2056",
                "extra_dims": "id_point=uint32"
            },
            {
                "type": "filters.crop", 
                "bounds": list_bounds,
            },
            {
                "type": "writers.las", 
                "filename": output_pattern, 
                'extra_dims': "id_point=uint32"
            }
        ]
    }
    if verbose:
        print("Creation of the stripes (might take a few minutes)")
    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    pipeline.execute()
    if verbose:
        print("Process done")


def remove_duplicates(laz_file, decimals=2):
    """
    Removes duplicate points from a LAS/LAZ file based on rounded 3D coordinates.

    Args:
        - laz_file (laspy.LasData): Input LAS/LAZ file as a laspy object.
        - decimals (int, optional): Number of decimals to round the coordinates for duplicate detection. Defaults to 2.

    Returns:
        - laspy.LasData: A new laspy object with duplicate points removed.
    """
        
    coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)).T, decimals)
    _, unique_indices = np.unique(coords, axis=0, return_index=True)
    mask = np.zeros(len(coords), dtype=bool)
    mask[unique_indices] = True

    # Create new LAS object
    header = laspy.LasHeader(point_format=laz_file.header.point_format, version=laz_file.header.version)
    new_las = laspy.LasData(header)

    for dim in laz_file.point_format.dimension_names:
        setattr(new_las, dim, getattr(laz_file, dim)[mask])

    return new_las


def match_pointclouds(laz1, laz2):
    """Sort laz2 to match the order of laz1 without changing laz1's order.

    Args:
        laz1: laspy.LasData object (reference order)
        laz2: laspy.LasData object (to be sorted)
    
    Returns:
        laz2 sorted to match laz1
    """
    # Retrieve and round coordinates for robust matching
    coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)), 2).T
    coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)), 2).T

    # Verify laz2 is of the same size as laz1
    assert len(coords_2) == len(coords_1), "laz2 should be a subset of laz1"

    # Create a dictionary mapping from coordinates to indices
    coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_1)}

    # Find indices in laz1 that correspond to laz2
    matching_indices = []
    failed = 0
    for coord in coords_2:
        try:
            matching_indices.append(coord_to_idx[tuple(coord)])
        except Exception as e:
            failed += 1

    matching_indices = np.array([coord_to_idx[tuple(coord)] for coord in coords_2])

    # Sort laz2 to match laz1
    sorted_indices = np.argsort(matching_indices)

    # Apply sorting to all attributes of laz2
    laz2.points = laz2.points[sorted_indices]

    return laz2  # Now sorted to match laz1


def flattening_tile(tile_src, tile_new_original_src, grid_size=10, method='cubic', do_save_floor=False, do_keep_existing=False, do_extrapolate_outside_hull=False, verbose=True):
    """
    Flattens a tile by interpolating the ground surface and subtracting it from the original elevation.

    Args:
        - tile_src (str): Path to the input tile in LAS/LAZ format.
        - tile_new_original_src (str): Path to save the resized original tile after filtering.
        - grid_size (int, optional): Size of the grid in meters for local interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to display progress and debug information. Defaults to True.

    Returns:
        - None: Saves the floor and flattened versions of the tile and updates the original file.
    """
    if os.path.exists(tile_new_original_src) and do_keep_existing:
        if verbose:
            print(f"Skipping. {tile_new_original_src} exists already")
        return
    
    # Load file
    laz = laspy.read(tile_src)
    init_len = len(laz)
    if init_len == 0:
        return
    
    if verbose:
        print(f"Removing duplicates: From {init_len} to {len(laz)}")
    
    points = np.vstack((laz.x, laz.y, laz.z)).T
    points_flatten = copy.deepcopy(points)
    points_interpolated = copy.deepcopy(points)

    # Divide into tiles and find local minimums
    x_min, y_min = np.min(points[:, :2], axis=0)
    x_max, y_max = np.max(points[:, :2], axis=0)

    x_bins = np.append(np.arange(x_min, x_max, grid_size), x_max)
    y_bins = np.append(np.arange(y_min, y_max, grid_size), y_max)

    grid = {i:{j:[] for j in range(y_bins.size - 1)} for i in range(x_bins.size -1)}
    for _, (px, py, pz) in tqdm(enumerate(points), total=len(points), desc="Creating grid", disable=verbose==False):
        xbin = np.clip(0, (px - x_min) // grid_size, x_bins.size - 2)
        ybin = np.clip(0, (py - y_min) // grid_size, y_bins.size - 2)
        try:
            grid[xbin][ybin].append((px, py, pz))
        except Exception as e:
            print(xbin)
            print(ybin)
            print(x_bins)
            print(y_bins)
            print(grid.keys())
            print(grid[0].keys())
            raise e


    # Create grid_min
    grid_used = np.zeros((x_bins.size - 1, y_bins.size - 1))
    lst_grid_min = []
    lst_grid_min_pos = []
    for x in grid.keys():
        for y in grid[x].keys():
            if np.array(grid[x][y]).shape[0] > 0:
                grid_used[x, y] = 1
                lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                arg_min = np.argmin(np.array(grid[x][y])[:,2])
                lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2])

                # test if border
                if x == list(grid.keys())[0]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [5, 0])
                if x == list(grid.keys())[-1]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [5, 0])
                if y == list(grid[x].keys())[0]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [0, 5])
                if y == list(grid[x].keys())[-1]:
                    lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                    lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [0, 5])
            else:
                grid_used[x, y] = 0
    arr_grid_min_pos = np.vstack(lst_grid_min_pos)
    if verbose:
        print("Resulting grid:")
        print(arr_grid_min_pos.shape)
        print(grid_used)

    # Interpolate
    points_xy = np.array(points)[:,0:2]
    if method == 'cubic':
        interpolated_min_z = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), points_xy, method="cubic", fill_value=-1)
    elif method == 'quadric':
        rbf = scipy.interpolate.Rbf(arr_grid_min_pos[:,0], arr_grid_min_pos[:,1], np.array(lst_grid_min), function='multiquadric', smooth=5)
        interpolated_min_z = rbf(points_xy[:,0], points_xy[:,1])
    else:
        raise ValueError("Wrong argument for method!")

    # Fill NaNs with nearest neighbor interpolation
    if do_extrapolate_outside_hull:
        nan_mask = interpolated_min_z == -1
        x = np.array(points)[:,0]
        y = np.array(points)[:,1]

        if np.any(nan_mask):
            interpolated_min_z[nan_mask] = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), (x[nan_mask], y[nan_mask]), method='nearest')

    mask_valid = np.array([x != -1 for x in list(interpolated_min_z)])
    points_interpolated = points_interpolated[mask_valid]
    points_interpolated[:, 2] = interpolated_min_z[mask_valid]

    if verbose:
        print("Interpolation:")
        print(f"Original number of points: {points.shape[0]}")
        print(f"Interpollated number of points: {points_interpolated.shape[0]} ({int(points_interpolated.shape[0] / points.shape[0]*100)}%)")

    # save floor
    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    new_las = laspy.LasData(header)

    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)
    setattr(new_las, 'x', points_interpolated[:,0])
    setattr(new_las, 'y', points_interpolated[:,1])
    setattr(new_las, 'z', points_interpolated[:,2])

    #   _Save new file
    if do_save_floor:
        new_las.write(tile_new_original_src.split('.laz')[0] + "_floor.laz")
        if verbose:
            print("Saved file: ", tile_new_original_src.split('.laz')[0] + "_floor.laz")

    # Flatten
    points_flatten = points_flatten[mask_valid]
    points_flatten[:,2] = points_flatten[:,2] - points_interpolated[:,2]

    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    header.point_count = 0
    new_las = laspy.LasData(header)


    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)

    setattr(new_las, 'x', points_flatten[:,0])
    setattr(new_las, 'y', points_flatten[:,1])
    setattr(new_las, 'z', points_flatten[:,2])

    #   _Save new file
    new_las.write(tile_new_original_src.split('.laz')[0] + "_flatten.laz")
    if verbose:
        print("Saved file: ", tile_new_original_src.split('.laz')[0] + "_flatten.laz")

    # Resize original file
    laz.points = laz.points[mask_valid]
    laz.write(tile_new_original_src)
    if verbose:
        print("Saved file: ", tile_new_original_src)


def flattening(src_tiles, src_new_tiles, grid_size=10, verbose=True, method='cubic', do_save_floor=True, do_keep_existing=False, verbose_full=False):
    """
    Applies the flattening process to all tiles in a directory using grid-based ground surface estimation.

    Args:
        - src_tiles (str): Path to the directory containing original tiles.
        - src_new_tiles (str): Path to the directory where resized tiles will be saved.
        - grid_size (int, optional): Size of the grid in meters for interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to show a general progress bar. Defaults to True.
        - verbose_full (bool, optional): Whether to print detailed info per tile. Defaults to False.

    Returns:
        - None: Processes and saves flattened tiles into their respective folders.
    """
    
    print("Starting flattening:")
    list_tiles = [x for x in os.listdir(src_tiles) if x.endswith('.laz')]
    for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles), desc="Processing", disable=verbose==False):
        if verbose_full:
            print("Flattening tile: ", tile)
        if do_keep_existing and os.path.exists(os.path.join(src_new_tiles, tile).split('.laz')[0] + "_flatten.laz"):
            continue

        flattening_tile(
            tile_src=os.path.join(src_tiles, tile), 
            tile_new_original_src=os.path.join(src_new_tiles, tile),
            grid_size=grid_size,
            method=method, 
            do_save_floor=do_save_floor,
            do_keep_existing=do_keep_existing,
            verbose=verbose_full,
            )
        

def merge_laz(list_files, output_file):
    """
    Merge multiple LAS/LAZ files into one using laspy.
    Preserves all dimensions including extra_dims like 'id_point'.
    """
    # Read the header from the first file
    first_las = laspy.read(list_files[0])
    header = laspy.LasHeader(point_format=first_las.header.point_format,
                              version=first_las.header.version)

    all_arrays = []
    for _, f in tqdm(enumerate(list_files), total=len(list_files)):
        las = laspy.read(f)
        all_arrays.append(las.points.array)  # extract structured array
        # print(f"Read {f} ({len(las)} pts)")

    # Concatenate structured arrays
    merged_array = np.concatenate(all_arrays)

    # Create new LasData with header
    out = laspy.LasData(header)

    # Wrap the concatenated array as ScaleAwarePointRecord and assign
    out.points = laspy.ScaleAwarePointRecord(
        merged_array,
        point_format=header.point_format,
        scales=header.scales,
        offsets=header.offsets
    )

    # Save
    out.write(output_file)
    print(f"Merged file saved to {output_file}")

### Splitting

In [7]:
# loading
start_preprocess_time = time()
laz_original = laspy.read(src_original)

In [26]:
src_with_id = src_original.split('.laz')[0] + "_with_id.laz"
src_folder_tiles_wo_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_no_overlap")
src_folder_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_overlap")
src_folder_flatten_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_flatten")
src_flatten_file = os.path.join(src_original.split('.laz')[0] + "_flatten.laz")
src_folder_stripes = os.path.join(os.path.dirname(src_flatten_file), os.path.basename(src_flatten_file).split('.laz')[0] + f"_stripes_{stripe_width}_m")


In [9]:
# adding an id to the points
id_point = np.arange(len(laz_original))
laz_original.add_extra_dim(laspy.ExtraBytesParams('id_point', type="uint32"))
laz_original.id_point = id_point

src_with_id = src_original.split('.laz')[0] + "_with_id.laz"
laz_original.write(src_with_id)

In [ ]:
# tiles without overlap
src_folder_tiles_wo_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_no_overlap")
tilling(src_with_id, src_folder_tiles_wo_overlap, tile_size)

Creation of tiles:


100%|██████████| 160/160 [03:29<00:00,  1.31s/it]


In [11]:
# tiles with overlap
src_folder_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_tiles_overlap")
tilling(src_with_id, src_folder_tiles_w_overlap, tile_size, overlap)

Creation of tiles:


100%|██████████| 160/160 [03:46<00:00,  1.41s/it]


### Flattening of stripes

In [36]:
# Flattening of tiles with overlap
src_folder_flatten_tiles_w_overlap = os.path.join(os.path.dirname(src_with_id), os.path.basename(src_with_id).split('.laz')[0] + "_flatten")
os.makedirs(src_folder_flatten_tiles_w_overlap, exist_ok=True)
flattening(
    src_tiles=src_folder_tiles_w_overlap,
    src_new_tiles=src_folder_flatten_tiles_w_overlap,
    grid_size=grid_size,
    method=method,
    do_save_floor=do_save_floor,
    do_keep_existing=do_keep_existing_flatten,
    verbose=True,
    verbose_full=False,
)

Starting flattening:


Processing:  40%|███▉      | 29/73 [26:39<40:26, 55.14s/it]   


MemoryError: Unable to allocate 45.4 GiB for an array with shape (7453854, 818) and data type float64

In [ ]:
# Creating flatten tiles w/o overlap and merging
def hash_coords(x, y, z, rounding=2):
    """Create a unique integer hash for each rounded coordinate triple."""
    return np.round(x, rounding) * 1e12 + np.round(y, rounding) * 1e6 + np.round(z, rounding)

list_flatten_to_merge = []
print("Creating flaten tiles without overlap")
# list_tiles = [x for x in os.listdir(src_folder_flatten_tiles_w_overlap) if not x.endswith('_flatten.laz')]
list_tiles = [x for x in os.listdir(src_folder_tiles_wo_overlap) if x.endswith('.laz')]
for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles)):
    # print(tile)
    assert os.path.exists(os.path.join(src_folder_tiles_wo_overlap, tile))

    laz_with_ov = laspy.read(os.path.join(src_folder_flatten_tiles_w_overlap, tile))
    laz_without_ov = laspy.read(os.path.join(src_folder_tiles_wo_overlap, tile))
    laz_flatten_with_ov = laspy.read(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten.laz"))

    mask = np.isin(laz_with_ov.id_point, laz_without_ov.id_point)

    laz_flatten_wo_ov = laz_flatten_with_ov[mask]
    laz_flatten_wo_ov.write(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten_no_ov.laz"))
    list_flatten_to_merge.append(os.path.join(src_folder_flatten_tiles_w_overlap, tile.split('.laz')[0] + "_flatten_no_ov.laz"))

# Merging
print("Merging all flatten tiles together (might take a few minutes)")
src_flatten_file = os.path.join(src_original.split('.laz')[0] + "_flatten.laz")
merge_laz(list_flatten_to_merge, src_flatten_file)



Creating flaten tiles without overlap


  0%|          | 0/58 [00:00<?, ?it/s]

100%|██████████| 58/58 [01:03<00:00,  1.09s/it]


Merging all flatten tiles together (might take a few minutes)


100%|██████████| 58/58 [00:08<00:00,  6.72it/s]


Merged file saved to D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\quadric_interp\Barmasse_2025_AllScans_Raw_Sub2cm_flatten.laz


In [ ]:
# Generate stripes from the merged flatten
src_folder_stripes = os.path.join(os.path.dirname(src_flatten_file), os.path.basename(src_flatten_file).split('.laz')[0] + f"_stripes_{stripe_width}_m")
x_span = laz_original.x.max() - laz_original.x.min()
y_span = laz_original.y.max() - laz_original.y.min()
dims = [stripe_width, y_span]

print("Creation of stripes:")
stripes_file(src_flatten_file, src_folder_stripes, dims, do_keep_existing=True)

Creation of stripes:


100%|██████████| 158/158 [03:19<00:00,  1.26s/it]


In [15]:
delta_time_loop = time() - start_preprocess_time
hours = int(delta_time_loop // 3600)
min = int((delta_time_loop - 3600 * hours) // 60)
sec = int(delta_time_loop - 3600 * hours - 60 * min)
print(f"==== Preprocess done in {hours}:{min}:{sec} ====")

==== Preprocess done in 1:6:29 ====


In [ ]:
# CLICK ON "Execute above cells" ON THIS CELL TO RUN THE PRE-PROCESS. OVER THERE (triangle with top arrow) --->

### Manual cleaning
Once the flattening step done, you will find the resulting stripes in the folder <>.

Please clean them and save them with the same name in the folder <> alredy created.

To be aware of regarding the cleaning:
 - a
 - b
 - c

Afterward, you can run the post-processing step.

### Post-process

In [19]:
src_clean_stripes = None

if src_clean_stripes == None:
    src_clean_stripes = src_folder_stripes

In [ ]:
# Merging
print("Merging all flatten tiles together (might take a few minutes)")
src_flatten_clean_file = os.path.join(src_original.split('.laz')[0] + "_flatten_clean.laz")
print(src_clean_stripes)
list_clean_flatten_to_merge = [os.path.join(src_clean_stripes, x) for x in os.listdir(src_clean_stripes) if x.endswith('laz')]

merge_laz(list_clean_flatten_to_merge, src_flatten_clean_file)


Merging all flatten tiles together (might take a few minutes)
D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\functional\Barmasse_2025_AllScans_Raw_Sub2cm_flatten_stripes_10_m


100%|██████████| 114/114 [00:06<00:00, 17.27it/s]


Merged file saved to D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\functional\Barmasse_2025_AllScans_Raw_Sub2cm_flatten_clean.laz


In [23]:
# remapping to original
# src = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test_flatten_clean.laz"
laz_flatten_clean = laspy.read(src_flatten_clean_file)
# print(src_flatten_file)
# print(list(laz_flatten_clean.point_format.dimension_names))
mask = np.isin(laz_original.id_point, laz_flatten_clean.id_point)
print(len(laz_flatten_clean))
print(len(laz_original))
print(mask.shape)
print(np.sum(mask))
src_final_result = src_original.split('.laz')[0] + "_FINAL.laz"
laz_original[mask].write(src_final_result)

85660348
101685240
(101685240,)
83926944


In [33]:
las = laspy.read(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test\Barmasse_2023_AllScans_Raw_Georef_with_id_tiles_overlap\Barmasse_2023_AllScans_Raw_Georef_with_id_tile_4.laz")
print(len(las))
print(len(set(las.id_point)))

76849043
76849043


In [9]:
import laspy
from pyproj import CRS
from laspy.vlrs.known import WktCoordinateSystemVlr


def ensure_crs_in_las(src_input, epsg=2056):
    las = laspy.read(src_input)
    if las.header.parse_crs() is None:
        crs = CRS.from_epsg(epsg)
        wkt = crs.to_wkt()
        vlr = WktCoordinateSystemVlr(wkt)
        las.header.vlrs.append(vlr)
        tmp_path = src_input.replace(".laz", "_crs.laz")
        las.write(tmp_path)
        print(f"✅ Added CRS {epsg} to {src_input}")
        return tmp_path
    else:
        return src_input


def tilling_slow(src_input, src_target, tile_size, overlap=0, verbose=True):
    """
    Tile the input LiDAR file into square tiles using PDAL and store them in the destination folder.

    Args:
        - verbose (bool): Whether to print verbose status updates.

    Returns:
        - None: Splits the input file into tiles and saves them in the destination folder.
    """

    print(f"Start tilling (with overlap = {overlap}m)...")
    os.makedirs(src_target, exist_ok=True)
    
    src_input = ensure_crs_in_las(src_input, epsg=2056)
    
    # compute the estimate number of tiles
    if verbose:
        print("Computing the estimated number of tiles...")
    original_file = laspy.read(src_input)
    x_min = original_file.x.min()
    x_max = original_file.x.max()
    y_min = original_file.y.min()
    y_max = original_file.y.max() 
    if verbose:
        print('Done!')


    
    # output_pattern = "tile_{i}_{j}.laz"
    output_pattern = os.path.join(
        src_target, 
        os.path.basename(src_input).split('.')[0] + "_tile_{i}_{j}.laz",
        )

    x_steps = int((x_max - x_min) / tile_size) + 1
    y_steps = int((y_max - y_min) / tile_size) + 1
    combinations = list(itertools.product(range(x_steps), range(y_steps)))
    for _, (i,j) in tqdm(enumerate(combinations), total=len(combinations)):
        x0 = x_min + i * tile_size - overlap
        x1 = x_min + (i + 1) * tile_size + overlap
        y0 = y_min + j * tile_size - overlap
        y1 = y_min + (j + 1) * tile_size + overlap

        bounds = f"([{x0},{x1}],[{y0},{y1}])"
        pipeline_json = {
            "pipeline": [
                # src_input,
                {
                    "type": "readers.las",
                    "filename": src_input,
                    "extra_dims": "id_point=uint32"
                },
                {
                    "type": "filters.reprojection",
                    "in_srs": "EPSG:2056",
                    "out_srs": "EPSG:2056"
                },
                {"type": "filters.crop", "bounds": bounds},
                {
                    "type": "writers.las",
                    "filename": output_pattern.format(i=i, j=j),
                    "extra_dims": "id_point=uint32"
                }
            ]
        }
        pipeline = pdal.Pipeline(json.dumps(pipeline_json))
        pipeline.execute()

In [ ]:
# def merge_las_files_inmemory(list_files, output_file):
#     """
#     Merge multiple LAS/LAZ files into one using laspy.
#     Preserves all dimensions including extra_dims like 'id_point'.
#     """
#     # Read the header from the first file
#     first_las = laspy.read(list_files[0])
#     header = laspy.LasHeader(point_format=first_las.header.point_format,
#                               version=first_las.header.version)

#     all_arrays = []
#     for f in list_files:
#         las = laspy.read(f)
#         all_arrays.append(las.points.array)  # extract structured array
#         print(f"Read {f} ({len(las)} pts)")

#     # Concatenate structured arrays
#     merged_array = np.concatenate(all_arrays)

#     # Create new LasData with header
#     out = laspy.LasData(header)

#     # Wrap the concatenated array as ScaleAwarePointRecord and assign
#     out.points = laspy.ScaleAwarePointRecord(
#         merged_array,
#         point_format=header.point_format,
#         scales=header.scales,
#         offsets=header.offsets
#     )

#     # Save
#     out.write(output_file)
#     print(f"Merged file saved to {output_file}")

In [ ]:
# def merge_las_files(list_files, output_path, verbose=True):
#     """
#     Merge multiple LAS/LAZ files into a single output file using laspy.

#     Args:
#         list_files (list[str]): Paths to input LAS/LAZ files.
#         output_path (str): Path to output merged LAS/LAZ file.
#         verbose (bool): Whether to print progress info.

#     Returns:
#         None
#     """
#     if len(list_files) == 0:
#         raise ValueError("No input files provided for merging.")

#     if verbose:
#         print(f"🧩 Merging {len(list_files)} LAS/LAZ files...")

#     # --- Read the first file to set the reference header ---
#     ref_las = laspy.read(list_files[0])
#     header = laspy.LasHeader(point_format=ref_las.header.point_format, version=ref_las.header.version)
#     header.offsets = ref_las.header.offsets
#     header.scales = ref_las.header.scales

#     # Copy CRS if available
#     if hasattr(ref_las.header, "epsg") and ref_las.header.epsg is not None:
#         header.epsg = ref_las.header.epsg

#     merged = laspy.LasData(header)
#     all_points = []

#     # --- Collect points from each file ---
#     for f in list_files:
#         las = laspy.read(f)
#         if verbose:
#             print(f"   → Reading {f} ({len(las)} points)")

#         # Ensure same point format
#         if las.header.point_format != ref_las.header.point_format:
#             raise ValueError(f"File {f} has different point format.")

#         # Convert to numpy structured array for fast stacking
#         all_points.append(las.points.array)

#     # --- Concatenate all points ---
#     merged.points = laspy.ScaleAwarePointRecord.merge(all_points, header=header)

#     # --- Save merged file ---
#     merged.write(output_path)
#     if verbose:
#         print(f"✅ Merged file saved to {output_path} ({len(merged)} points)")

In [ ]:
# import laspy
# import numpy as np
# import os

# def merge_las_files_inmemory(list_files, output_path, verbose=True):
#     """
#     Merge multiple LAS/LAZ files into one using laspy, preserving extra dimensions and CRS.
#     """
#     if len(list_files) == 0:
#         raise ValueError("No input files provided for merging.")

#     # --- Read reference file ---
#     ref = laspy.read(list_files[0])
#     header = laspy.LasHeader(point_format=ref.header.point_format, version=ref.header.version)
#     header.scales = ref.header.scales
#     header.offsets = ref.header.offsets

#     # Copy CRS/VLRs if available
#     header.vlrs = list(ref.header.vlrs)
#     try:
#         crs = ref.header.parse_crs()
#         if crs is not None:
#             header.parse_crs(crs)
#     except Exception:
#         pass

#     # --- Prepare dimension names ---
#     dim_names = ref.point_format.dimension_names
#     if verbose:
#         print(f"Detected dimensions: {dim_names}")

#     # --- Concatenate all points ---
#     all_points = []
#     total_points = 0
#     for f in list_files:
#         las = laspy.read(f)
#         if las.point_format != ref.point_format:
#             raise ValueError(f"Point format mismatch in {f}")
#         all_points.append(las.points)
#         total_points += len(las)
#         if verbose:
#             print(f"Read {f} ({len(las)} pts)")

#     # Extract structured arrays and concatenate
#     arrays_to_merge = [p.array for p in all_points]
#     merged_array = np.concatenate(arrays_to_merge)

#     # Wrap back as ScaleAwarePointRecord
#     # merged_points = laspy.ScaleAwarePointRecord(merged_array, header=header)

#     # Assign to LasData
#     out = laspy.LasData(header)
#     out.points = merged_array

#     # # Stack ScaleAwarePointRecords
#     # merged_points = np.concatenate(all_points)

#     # # --- Create LasData and assign points ---
#     # out = laspy.LasData(header)
#     # out.points = laspy.ScaleAwarePointRecord(merged_points, header=header)

#     # --- Write output ---
#     out.write(output_path)
#     if verbose:
#         print(f"✅ Merged {len(list_files)} files ({total_points} pts) -> {output_path}")


In [ ]:
# import sys
# import traceback
# import laspy
# import os
# try:
#     print('Running Merge LAS')

#     #This is the las file to append to.  DO NOT STORE THIS FILE IN THE SAME DIRECTORY AS BELOW...
#     out_las = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\functional\test_merge\merged_file.laz "  
#     #this is a directory of las files
#     inDir = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2025\functional\test_merge\source"  

#     def append_to_las(in_laz, out_las):
#         with laspy.open(out_las, mode='a') as outlas:
#             with laspy.open(in_las) as inlas:
#                 for points in inlas.chunk_iterator(2_000_000):
#                     outlas.append_points(points)

#     # print(list(os.listdir(inDir)))
#     for (dirpath, dirnames, filenames) in os.walk(inDir):
#         # print(filenames)
#         for inFile in filenames:
#             if inFile.endswith('.laz'):
#                 in_las = os.path.join(dirpath, inFile)
#                 print(in_las)
#                 append_to_las(in_las, out_las)
        
#     print('Finished without errors - merge_LAS.py')
# except:
#     tb = sys.exc_info()[2]
#     tbinfo = traceback.format_tb(tb)[0]
#     print('Error in append las')
#     print ("PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError     Info:\n" + str(sys.exc_info()[1]))  

In [ ]:
# import laspy
# import numpy as np
# import os

# def crop_laspy(src_input, src_target, tile_size, overlap=0, verbose=True):
#     """
#     Crops a LAS/LAZ file into tiles using laspy directly (preserves id_point perfectly).
#     """
#     os.makedirs(src_target, exist_ok=True)

#     las = laspy.read(src_input)
#     x_min, x_max = las.x.min(), las.x.max()
#     y_min, y_max = las.y.min(), las.y.max()

#     x_steps = int((x_max - x_min) / tile_size) + 1
#     y_steps = int((y_max - y_min) / tile_size) + 1

#     for ix in range(x_steps):
#         for iy in range(y_steps):
#             x0 = x_min + ix * tile_size - overlap
#             x1 = x_min + (ix + 1) * tile_size + overlap
#             y0 = y_min + iy * tile_size - overlap
#             y1 = y_min + (iy + 1) * tile_size + overlap

#             mask = (
#                 (las.x >= x0) & (las.x <= x1) &
#                 (las.y >= y0) & (las.y <= y1)
#             )
#             if not np.any(mask):
#                 continue

#             tile = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
#             tile.points = las.points[mask]

#             tile_filename = os.path.join(
#                 src_target,
#                 f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
#             )
#             tile.write(tile_filename)
#             if verbose:
#                 print(f"✅ Wrote {tile_filename} ({mask.sum()} points)")



# def crop_laspy(src_input, src_target, tile_size, overlap=0, verbose=True):
#     """
#     Crops a LAS/LAZ file into tiles using laspy directly (preserves id_point and coordinates).
#     """
#     os.makedirs(src_target, exist_ok=True)

#     las = laspy.read(src_input)
#     x_min, x_max = las.x.min(), las.x.max()
#     y_min, y_max = las.y.min(), las.y.max()

#     x_steps = int((x_max - x_min) / tile_size) + 1
#     y_steps = int((y_max - y_min) / tile_size) + 1

#     for ix in range(x_steps):
#         for iy in range(y_steps):
#             x0 = x_min + ix * tile_size - overlap
#             x1 = x_min + (ix + 1) * tile_size + overlap
#             y0 = y_min + iy * tile_size - overlap
#             y1 = y_min + (iy + 1) * tile_size + overlap

#             mask = (
#                 (las.x >= x0) & (las.x <= x1) &
#                 (las.y >= y0) & (las.y <= y1)
#             )
#             if not np.any(mask):
#                 continue

#             # ✅ Create new file with proper header scale/offset
#             header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
#             header.offsets = las.header.offsets
#             header.scales = las.header.scales

#             # ✅ Copy CRS if any
#             if hasattr(las.header, "epsg") and las.header.epsg is not None:
#                 header.epsg = las.header.epsg

#             tile = laspy.LasData(header)
#             tile.points = las.points[mask]

#             tile_filename = os.path.join(
#                 src_target,
#                 f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
#             )
#             tile.write(tile_filename)

#             if verbose:
#                 print(f"✅ Wrote {tile_filename} ({mask.sum()} points)")


In [18]:
# tilling_slow(src_with_id, src_folder_tiles_w_overlap, tile_size, overlap)
crop_laspy(src_with_id, src_folder_tiles_w_overlap, tile_size, overlap=0, verbose=True)

✅ Wrote D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tiles_overlap\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tile_0_0.laz (11986946 points)
✅ Wrote D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tiles_overlap\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tile_0_1.laz (5450402 points)
✅ Wrote D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tiles_overlap\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tile_1_0.laz (57045997 points)
✅ Wrote D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tiles_overlap\Barmasse_2025_AllScans_Raw_Sub2cm_with_id_tile_1_1.laz (8776736 points)
✅ Wrote D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2023\test2\Barmasse_2025_AllScans_Raw_Sub2

In [ ]:
# # utils
# def tilling(src_input, src_target, tile_size, overlap=0, verbose=True):
#     """
#     Tile the input LiDAR file into square tiles using PDAL and store them in the destination folder.

#     Args:
#         - verbose (bool): Whether to print verbose status updates.

#     Returns:
#         - None: Splits the input file into tiles and saves them in the destination folder.
#     """

#     print(f"Start tilling (with overlap = {overlap}m)...")
#     os.makedirs(src_target, exist_ok=True)
    
#     # compute the estimate number of tiles
#     if verbose:
#         print("Computing the estimated number of tiles...")
#     original_file = laspy.read(src_input)
#     x_min = original_file.x.min()
#     x_max = original_file.x.max()
#     y_min = original_file.y.min()
#     y_max = original_file.y.max() 
#     if verbose:
#         print('Done!')


    
#     # output_pattern = "tile_{i}_{j}.laz"
#     output_pattern = os.path.join(
#         src_target, 
#         os.path.basename(src_input).split('.')[0] + "_tile_{i}_{j}.laz",
#         )

#     x_steps = int((x_max - x_min) / tile_size) + 1
#     y_steps = int((y_max - y_min) / tile_size) + 1
#     combinations = list(itertools.product(range(x_steps), range(y_steps)))
#     for _, (i,j) in tqdm(enumerate(combinations), total=len(combinations)):
#         x0 = x_min + i * tile_size - overlap
#         x1 = x_min + (i + 1) * tile_size + overlap
#         y0 = y_min + j * tile_size - overlap
#         y1 = y_min + (j + 1) * tile_size + overlap

#         bounds = f"([{x0},{x1}],[{y0},{y1}])"
#         pipeline_json = {
#             "pipeline": [
#                 # src_input,
#                 {
#                     "type": "readers.las",
#                     "filename": src_input,
#                     "extra_dims": "id_point=uint32"
#                 },
#                 {"type": "filters.crop", "bounds": bounds},
#                 {
#                     "type": "writers.las",
#                     "filename": output_pattern.format(i=i, j=j),
#                     "extra_dims": "id_point=uint32"
#                 }
#             ]
#         }

#         pipeline = pdal.Pipeline(json.dumps(pipeline_json))
#         pipeline.execute()


# def stripes_file(src_input_file, src_output, dims, do_keep_existant=False):
#     [tile_size_x, tile_size_y] = dims
#     laz = laspy.read(src_input_file)
#     # output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_stripes_{tile_size_x}_{tile_size_y}")
#     os.makedirs(src_output, exist_ok=True)

#     xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

#     x_edges = np.arange(xmin, xmax, tile_size_x)
#     y_edges = np.arange(ymin, ymax, tile_size_y)


#     for i, x0 in enumerate(x_edges):
#         print(f"Column {i+1} / {len(x_edges)}:")
#         for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
#             output = os.path.join(src_output,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
#             if do_keep_existant and os.path.exists(output):
#                 continue
#             x1 = x0 + tile_size_x
#             y1 = y0 + tile_size_y
#             bounds = f"([{x0},{x1}],[{y0},{y1}])"
#             pipeline_json = {
#                 "pipeline": [
#                     src_input_file,
#                     {"type": "filters.crop", "bounds": bounds},
#                     {"type": "writers.las", "filename": output, 'extra_dims': 'all'}
#                 ]
#             }

#             pipeline = pdal.Pipeline(json.dumps(pipeline_json))
#             pipeline.execute()


# def stripes_file(src_input_file, src_output, dims, do_keep_existant=False):
#     [tile_size_x, tile_size_y] = dims
#     laz = laspy.read(src_input_file)
#     # output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_stripes_{tile_size_x}_{tile_size_y}")
#     os.makedirs(src_output, exist_ok=True)

#     xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

#     x_edges = np.arange(xmin, xmax, tile_size_x)
#     y_edges = np.arange(ymin, ymax, tile_size_y)


#     for i, x0 in enumerate(x_edges):
#         print(f"Column {i+1} / {len(x_edges)}:")
#         for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
#             output = os.path.join(src_output,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
#             if do_keep_existant and os.path.exists(output):
#                 continue
#             x1 = x0 + tile_size_x
#             y1 = y0 + tile_size_y
#             bounds = f"([{x0},{x1}],[{y0},{y1}])"
#             pipeline_json = {
#                 "pipeline": [
#                     src_input_file,
#                     {"type": "filters.crop", "bounds": bounds},
#                     {"type": "writers.las", "filename": output, 'extra_dims': 'all'}
#                 ]
#             }

#             pipeline = pdal.Pipeline(json.dumps(pipeline_json))
#             pipeline.execute()



# def remove_duplicates(laz_file, decimals=2):
#     """
#     Removes duplicate points from a LAS/LAZ file based on rounded 3D coordinates.

#     Args:
#         - laz_file (laspy.LasData): Input LAS/LAZ file as a laspy object.
#         - decimals (int, optional): Number of decimals to round the coordinates for duplicate detection. Defaults to 2.

#     Returns:
#         - laspy.LasData: A new laspy object with duplicate points removed.
#     """
        
#     coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)).T, decimals)
#     _, unique_indices = np.unique(coords, axis=0, return_index=True)
#     mask = np.zeros(len(coords), dtype=bool)
#     mask[unique_indices] = True

#     # Create new LAS object
#     header = laspy.LasHeader(point_format=laz_file.header.point_format, version=laz_file.header.version)
#     new_las = laspy.LasData(header)

#     for dim in laz_file.point_format.dimension_names:
#         setattr(new_las, dim, getattr(laz_file, dim)[mask])

#     return new_las

# def match_pointclouds(laz1, laz2):
#     """Sort laz2 to match the order of laz1 without changing laz1's order.

#     Args:
#         laz1: laspy.LasData object (reference order)
#         laz2: laspy.LasData object (to be sorted)
    
#     Returns:
#         laz2 sorted to match laz1
#     """
#     # Retrieve and round coordinates for robust matching
#     coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)), 2).T
#     coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)), 2).T

#     # Verify laz2 is of the same size as laz1
#     assert len(coords_2) == len(coords_1), "laz2 should be a subset of laz1"

#     # Create a dictionary mapping from coordinates to indices
#     coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_1)}

#     # Find indices in laz1 that correspond to laz2
#     matching_indices = []
#     failed = 0
#     for coord in coords_2:
#         try:
#             matching_indices.append(coord_to_idx[tuple(coord)])
#         except Exception as e:
#             failed += 1

#     matching_indices = np.array([coord_to_idx[tuple(coord)] for coord in coords_2])

#     # Sort laz2 to match laz1
#     sorted_indices = np.argsort(matching_indices)

#     # Apply sorting to all attributes of laz2
#     laz2.points = laz2.points[sorted_indices]

#     return laz2  # Now sorted to match laz1


# def flattening_tile(tile_src, tile_new_original_src, grid_size=10, verbose=True):
#     """
#     Flattens a tile by interpolating the ground surface and subtracting it from the original elevation.

#     Args:
#         - tile_src (str): Path to the input tile in LAS/LAZ format.
#         - tile_new_original_src (str): Path to save the resized original tile after filtering.
#         - grid_size (int, optional): Size of the grid in meters for local interpolation. Defaults to 10.
#         - verbose (bool, optional): Whether to display progress and debug information. Defaults to True.

#     Returns:
#         - None: Saves the floor and flattened versions of the tile and updates the original file.
#     """

#     # Load file
#     laz = laspy.read(tile_src)
#     init_len = len(laz)
#     if init_len == 0:
#         return
#     # laz = remove_duplicates(laz)
#     if verbose:
#         print(f"Removing duplicates: From {init_len} to {len(laz)}")
#     # laz.write(tile_new_original_src)
    
#     points = np.vstack((laz.x, laz.y, laz.z)).T
#     points_flatten = copy.deepcopy(points)
#     points_interpolated = copy.deepcopy(points)

#     # Divide into tiles and find local minimums
#     x_min, y_min = np.min(points[:, :2], axis=0)
#     x_max, y_max = np.max(points[:, :2], axis=0)

#     x_bins = np.append(np.arange(x_min, x_max, grid_size), x_max)
#     y_bins = np.append(np.arange(y_min, y_max, grid_size), y_max)

#     grid = {i:{j:[] for j in range(y_bins.size - 1)} for i in range(x_bins.size -1)}
#     for _, (px, py, pz) in tqdm(enumerate(points), total=len(points), desc="Creating grid", disable=verbose==False):
#         xbin = np.clip(0, (px - x_min) // grid_size, x_bins.size - 2)
#         ybin = np.clip(0, (py - y_min) // grid_size, y_bins.size - 2)
#         try:
#             grid[xbin][ybin].append((px, py, pz))
#         except Exception as e:
#             print(xbin)
#             print(ybin)
#             print(x_bins)
#             print(y_bins)
#             print(grid.keys())
#             print(grid[0].keys())
#             raise e


#     # Create grid_min
#     grid_used = np.zeros((x_bins.size - 1, y_bins.size - 1))
#     lst_grid_min = []
#     lst_grid_min_pos = []
#     for x in grid.keys():
#         for y in grid[x].keys():
#             if np.array(grid[x][y]).shape[0] > 0:
#                 grid_used[x, y] = 1
#                 lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
#                 arg_min = np.argmin(np.array(grid[x][y])[:,2])
#                 lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2])

#                 # test if border
#                 if x == list(grid.keys())[0]:
#                     lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
#                     lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [5, 0])
#                 if x == list(grid.keys())[-1]:
#                     lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
#                     lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [5, 0])
#                 if y == list(grid[x].keys())[0]:
#                     lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
#                     lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] - [0, 5])
#                 if y == list(grid[x].keys())[-1]:
#                     lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
#                     lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2] + [0, 5])
#             else:
#                 grid_used[x, y] = 0
#     arr_grid_min_pos = np.vstack(lst_grid_min_pos)
#     if verbose:
#         print("Resulting grid:")
#         print(arr_grid_min_pos.shape)
#         print(grid_used)

#     # Interpolate
#     points_xy = np.array(points)[:,0:2]
#     interpolated_min_z = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), points_xy, method="cubic", fill_value=-1)

#     # # Fill NaNs with nearest neighbor interpolation
#     # nan_mask = np.isnan(interpolated_min_z)
#     # x = np.array(points)[:,0]
#     # y = np.array(points)[:,1]
#     # z = np.array(points)[:,2]

#     # if np.any(nan_mask):
#     #     interpolated_min_z[nan_mask] = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), (x[nan_mask], y[nan_mask]), method='nearest')

#     mask_valid = np.array([x != -1 for x in list(interpolated_min_z)])
#     points_interpolated = points_interpolated[mask_valid]
#     points_interpolated[:, 2] = interpolated_min_z[mask_valid]

#     if verbose:
#         print("Interpolation:")
#         print(f"Original number of points: {points.shape[0]}")
#         print(f"Interpollated number of points: {points_interpolated.shape[0]} ({int(points_interpolated.shape[0] / points.shape[0]*100)}%)")

#     # save floor
#     filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
#     header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
#     new_las = laspy.LasData(header)

#     #   _Assign filtered and modified data
#     for dim, values in filtered_points.items():
#         setattr(new_las, dim, values)
#     setattr(new_las, 'x', points_interpolated[:,0])
#     setattr(new_las, 'y', points_interpolated[:,1])
#     setattr(new_las, 'z', points_interpolated[:,2])

#     #   _Save new file
#     # floor_dir = os.path.join(os.path.dirname(tile_src), 'floor')
#     # os.makedirs(floor_dir, exist_ok=True)
#     # new_las.write(os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))
#     # if verbose:
#     #     print("Saved file: ", os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))

#     # Flatten
#     points_flatten = points_flatten[mask_valid]
#     points_flatten[:,2] = points_flatten[:,2] - points_interpolated[:,2]

#     filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
#     header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
#     header.point_count = 0
#     new_las = laspy.LasData(header)


#     #   _Assign filtered and modified data
#     for dim, values in filtered_points.items():
#         setattr(new_las, dim, values)

#     setattr(new_las, 'x', points_flatten[:,0])
#     setattr(new_las, 'y', points_flatten[:,1])
#     setattr(new_las, 'z', points_flatten[:,2])

#     #   _Save new file
#     # flatten_dir = os.path.join(os.path.dirname(tile_src), 'flatten')
#     # os.makedirs(flatten_dir, exist_ok=True)
#     # new_las.write(os.path.join(os.path.dirname(tile_src), os.path.basename(tile_src).split('.laz')[0] + f"_flatten_{grid_size}m.laz"))
#     new_las.write(tile_new_original_src.split('.laz')[0] + "_flatten.laz")
#     if verbose:
#         print("Saved file: ", tile_new_original_src.split('.laz')[0] + "_flatten.laz")

#     # Resize original file
#     laz.points = laz.points[mask_valid]
#     laz.write(tile_new_original_src)
#     if verbose:
#         print("Saved file: ", tile_new_original_src)


# def flattening(src_tiles, src_new_tiles, grid_size=10, verbose=True, verbose_full=False):
#     """
#     Applies the flattening process to all tiles in a directory using grid-based ground surface estimation.

#     Args:
#         - src_tiles (str): Path to the directory containing original tiles.
#         - src_new_tiles (str): Path to the directory where resized tiles will be saved.
#         - grid_size (int, optional): Size of the grid in meters for interpolation. Defaults to 10.
#         - verbose (bool, optional): Whether to show a general progress bar. Defaults to True.
#         - verbose_full (bool, optional): Whether to print detailed info per tile. Defaults to False.

#     Returns:
#         - None: Processes and saves flattened tiles into their respective folders.
#     """
    
#     print("Starting flattening:")
#     list_tiles = [x for x in os.listdir(src_tiles) if x.endswith('.laz')]
#     for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles), desc="Processing", disable=verbose==False):
#         if verbose_full:
#             print("Flattening tile: ", tile)
#         flattening_tile(
#             tile_src=os.path.join(src_tiles, tile), 
#             tile_new_original_src=os.path.join(src_new_tiles, tile),
#             grid_size=grid_size,
#             verbose=verbose_full,
#             )


In [ ]:
# def tilling(src_input, src_target, tile_size, overlap=0, verbose=True):
#     """
#     Tile the input LiDAR file into square tiles using PDAL and store them in the destination folder.

#     Args:
#         - verbose (bool): Whether to print verbose status updates.

#     Returns:
#         - None: Splits the input file into tiles and saves them in the destination folder.
#     """

#     print(f"Start tilling (with overlap = {overlap}m)...")
#     os.makedirs(src_target, exist_ok=True)

#     # Compute the bounds from the input file
#     if verbose:
#         print("Computing the bounds...")
#     original_file = laspy.read(src_input)
#     x_min = original_file.x.min()
#     x_max = original_file.x.max()
#     y_min = original_file.y.min()
#     y_max = original_file.y.max()
#     if verbose:
#         print(f"Done! Bounding box: x=[{x_min:.2f}, {x_max:.2f}], y=[{y_min:.2f}, {y_max:.2f}]")

#     # Compute tile steps
#     x_steps = int((x_max - x_min) / tile_size) + 1
#     y_steps = int((y_max - y_min) / tile_size) + 1
#     combinations = list(itertools.product(range(x_steps), range(y_steps)))

#     if verbose:
#         print(f"Creating {len(combinations)} tiles...")

#     # Loop over each tile (avoid list of bounds)
#     for i, (ix, iy) in enumerate(tqdm(combinations, total=len(combinations))):
#         x0 = x_min + ix * tile_size - overlap
#         x1 = x_min + (ix + 1) * tile_size + overlap
#         y0 = y_min + iy * tile_size - overlap
#         y1 = y_min + (iy + 1) * tile_size + overlap

#         bounds = f"([{x0},{x1}],[{y0},{y1}])"

#         tile_filename = os.path.join(
#             src_target,
#             f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
#         )

#         pipeline_json = {
#             "pipeline": [
#                 {
#                     "type": "readers.las",
#                     "filename": src_input,
#                     "extra_dims": "id_point=uint32"
#                 },
#                 {
#                     "type": "filters.crop",
#                     "bounds": bounds
#                 },
#                 {
#                     "type": "writers.las",
#                     "filename": tile_filename,
#                     "extra_dims": "id_point=uint32"
#                 }
#             ]
#         }

#         pipeline = pdal.Pipeline(json.dumps(pipeline_json))
#         pipeline.execute()

#     if verbose:
#         print("✅ All tiles successfully created.")

In [ ]:
# # import json
# # import laspy
# # import os
# # import itertools
# # import pdal

# def tilling_batch(src_input, src_target, tile_size, overlap=0, verbose=True):
#     """
#     Tile a LAS/LAZ file into multiple tiles in one PDAL pipeline execution,
#     avoiding reloading the dataset multiple times.
#     """

#     os.makedirs(src_target, exist_ok=True)

#     original_file = laspy.read(src_input)
#     x_min, x_max = original_file.x.min(), original_file.x.max()
#     y_min, y_max = original_file.y.min(), original_file.y.max()

#     x_steps = int((x_max - x_min) / tile_size) + 1
#     y_steps = int((y_max - y_min) / tile_size) + 1

#     combinations = list(itertools.product(range(x_steps), range(y_steps)))

#     # we’ll accumulate crop/writer pairs (no repeated readers)
#     branches = []
#     for ix, iy in combinations:
#         x0 = x_min + ix * tile_size - overlap
#         x1 = x_min + (ix + 1) * tile_size + overlap
#         y0 = y_min + iy * tile_size - overlap
#         y1 = y_min + (iy + 1) * tile_size + overlap

#         bounds = f"([{x0},{x1}],[{y0},{y1}])"
#         tile_filename = os.path.join(
#             src_target,
#             f"{os.path.splitext(os.path.basename(src_input))[0]}_tile_{ix}_{iy}.laz"
#         )

#         branches.extend([  # ✅ use extend to flatten directly
#             {"type": "filters.crop", "bounds": bounds},
#             {"type": "writers.las", "filename": tile_filename, "extra_dims": "id_point=uint32"}
#         ])

#     # ✅ Only one reader at the top, then all crops/writers
#     pipeline_json = {
#         "pipeline": [
#             {"type": "readers.las", "filename": src_input, "extra_dims": "id_point=uint32"},
#             *branches  # expands list to flat sequence
#         ]
#     }

#     if verbose:
#         print(f"Running PDAL with {len(combinations)} tile branches...")

#     pipeline = pdal.Pipeline(json.dumps(pipeline_json))
#     pipeline.execute()

#     if verbose:
#         print("✅ Tiling complete.")


